# 준비

## 라이브러리 호출

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import callbacks, layers, models, optimizers, losses, applications
from tensorflow.keras.layers.experimental import preprocessing

## 데이터 셋 로드

In [2]:
batch_size = 32
IMG_SIZE = 224

In [3]:
train_ds = image_dataset_from_directory("D:/Dataset/랜드마크 이미지/Training",
                                        validation_split=0.15,
                                        subset="training",
                                        seed=123,
                                        image_size=(IMG_SIZE, IMG_SIZE),
                                        batch_size=batch_size)

valid_ds = image_dataset_from_directory("D:/Dataset/랜드마크 이미지/Training",
                                        validation_split=0.15,
                                        subset="validation",
                                        seed=123,
                                        image_size=(IMG_SIZE, IMG_SIZE),
                                        batch_size=batch_size)

test_ds = image_dataset_from_directory("D:/Dataset/랜드마크 이미지/Validation",
                                        image_size=(IMG_SIZE, IMG_SIZE),
                                        batch_size=batch_size)

Found 12396 files belonging to 84 classes.
Using 10537 files for training.
Found 12396 files belonging to 84 classes.
Using 1859 files for validation.
Found 1504 files belonging to 84 classes.


In [4]:
class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print(class_names, NUM_CLASSES)

['6.25격전지개미고개', 'BRT작은미술관', '가락마을20단지호반베르디움5차아파트', '가락마을3단지호반베르디움4차아파트', '가온마을1단지힐스테이트세종2차', '가재마을세종호반베르디움2단지아파트', '고운뜰전망대', '국립세종도서관', '국립세종수목원', '국립조세박물관', '금강수목원', '금강수변공원', '금강자연휴양림', '금남남산영당', '금남모인당', '금남문절사', '김종서장군묘소', '나성독락정', '덕성서원', '덕천군사우', '도램마을10단지호반베르디움어반시티아파트', '도램마을15단지힐스테이트아파트', '밀양박씨오충정려', '박산리작약골쌍탑제', '범지기마을10단지푸르지오아파트', '범지기마을7단지호반베르디움에코시티아파트', '범지기마을8단지푸르지오아파트', '베어트리파크', '병산사', '부강성당', '부강초등학교강당', '비암사', '새뜸1단지메이저시티푸르지오', '새뜸마을10단지더샵힐스테이트아파트', '새뜸마을11단지더샵힐스테이트아파트', '새뜸마을14단지더샵힐스테이트아파트', '새뜸마을5단지아이파크메이저아파트', '새뜸마을6단지힐스테이트메이저아파트', '성요한성당', '세종e편한세상푸르지오아파트', '세종고속시외버스터미널', '세종대곡리삼층석탑', '세종시립민속박물관', '세종전통장류박물관', '세종특별자치시마을기록문화관', '세종행복도시홍보관', '세종호수공원', '세종힐스테이트3차', '수루배마을4단지더샵예미지', '수루배마을6단지세종더샵예미지', '연기대첩비공원', '연기봉산동의향나무', '연기향교', '연기향토박물관', '연동송용리마애여래입상', '연화사', '영평사', '은하수공원', '자성사', '전의비암사극락보전', '전의비암사삼층석탑', '전의향교', '조천연꽃공원', '조치원문화정원', '조치원봉산영당', '조치원신흥e편한세상아파트', '조치원신흥푸르지오아파트', '조치원역경부선', '조치원자이아파트', '조치원죽림푸르지오아파트', '천주교대전교구세종성프란치스코성당', '첫마을4단지푸르지오아파트', '첫마을5단지

## 데이터셋 시각화

In [5]:
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")

NameError: name 'plt' is not defined

In [24]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(64, 180, 180, 3)
(64,)


## 데이터 셋 성능강화 및 증강, 라벨 원-핫

In [6]:
def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.map(input_preprocess)
train_ds = train_ds.prefetch(AUTOTUNE)
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(1000)

valid_ds = valid_ds.map(input_preprocess).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.map(input_preprocess).cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
def img_augmentation(x):
    y = preprocessing.Rescaling(scale=1.0/255.0)(x)
    y = preprocessing.RandomRotation(0.2)(y)
    y = preprocessing.RandomZoom(0.1)(y)
    y = preprocessing.RandomFlip("horizontal")(y)
    y = preprocessing.RandomHeight(0.1)(y)
    y = preprocessing.RandomWidth(0.1)(y)
    y = preprocessing.RandomContrast(factor=0.1)(y)
    return y

# 모델링

## 콜백

In [11]:
ckp = callbacks.ModelCheckpoint(
    "model_2.h5",  # 경로
    save_best_only=True,  # 가장 높은 점수의 모델만
    monitor="val_acc")  # val_loss 를 보고 판단
early_stopping = callbacks.EarlyStopping(
    monitor="val_acc",
    patience=7)  # 7번까지는 기다릴수있다(낮아지는 횟수가 아니라 최고점 이후 점수가 상승하지 않는 횟수)
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor="val_acc",
    patience=4,
    facotr=0.1  # 학습률을 1/10 으로 줄임
)

In [9]:
base_model = applications.EfficientNetB0(input_shape=(IMG_SIZE,IMG_SIZE,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False # 베이스 모델은 학습하지 않음

In [10]:
def build_eff_model():
    # Top
    x = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    y = base_model(x, training=False)

    # Rebuild
    y = layers.GlobalAveragePooling2D(name="avg_pool")(y)
    y = layers.BatchNormalization()(y)
    top_dropout_rate = 0.2
    y = layers.Dropout(top_dropout_rate, name="top_dropout")(y)
    y = layers.Dense(84, name="Pred")(y)
    y = layers.PReLU()(y)
    y = layers.Softmax(dtype="float32", name="softmax")(y)
    
    # compile
    model = models.Model(x, y, name="EfficientNet")
    optimizer = optimizers.Adam(learning_rate=0.01)
    loss = losses.CategoricalCrossentropy()
    model.compile(optimizer=optimizer, loss=loss, metrics=["acc"])
    return model
model = build_eff_model()

In [12]:
def build_eff_model():
    # Top
    x = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    y = base_model(x, training=False)

    # Rebuild
    y = layers.GlobalAveragePooling2D(name="avg_pool")(y)
    y = layers.BatchNormalization()(y)
    top_dropout_rate = 0.2
    y = layers.Dropout(top_dropout_rate, name="top_dropout")(y)
    y = layers.Dense(NUM_CLASSES, activation="softmax", name="Pred")(y)
    
    # compile
    model = models.Model(x, y, name="EfficientNet")
    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"])
    return model
model = build_eff_model()

In [ ]:
history = model.fit(train_ds,
                    epochs=30,
                    validation_data=valid_ds,
                    callbacks=[ckp, early_stopping, reduce_lr])

Epoch 1/30
